In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

np.random.seed(123)
torch.manual_seed(123)

In [2]:
# Custom loss function
class SquaredHingeLoss(nn.Module):
    def forward(self, predicted, a, b):
        loss = torch.relu(predicted - b) + torch.relu(a - predicted)
        return torch.sum(loss**2)

In [3]:
# train data
data_df   = pd.read_csv('data/seq_stat.csv')
target_df = pd.read_csv('record/data_fold1.csv')

data      = data_df.iloc[:, 1:].to_numpy()
targets_1 = target_df.iloc[:, 1:2].to_numpy()
targets_2 = target_df.iloc[:, 2:3].to_numpy()

data      = torch.FloatTensor(data)
targets_1 = torch.FloatTensor(targets_1)
targets_2 = torch.FloatTensor(targets_2)

# test_data
target_test_df = pd.read_csv('record/data_fold2.csv')
targets_test_1 = target_test_df.iloc[:, 1:2].to_numpy()
targets_test_2 = target_test_df.iloc[:, 2:3].to_numpy()

targets_test_1 = torch.FloatTensor(targets_test_1)
targets_test_2 = torch.FloatTensor(targets_test_2)

In [4]:
# Define the deep learning model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(9, 12)
        self.fc2 = nn.Linear(12, 12)
        self.fc3 = nn.Linear(12, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# Instantiate the model, define custom loss function, and optimizer
model = MyModel()
squared_hinge_loss = SquaredHingeLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
epochs = 10000
for epoch in range(epochs+1):
    # Forward pass
    outputs = model(data)
    
    # Compute the custom loss
    loss_train = squared_hinge_loss(outputs, targets_1, targets_2)
    loss_test  = squared_hinge_loss(outputs, targets_test_1, targets_test_2)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    
    # Print the loss every 100 epochs
    if (epoch) % 500 == 0:
        print(f'Epoch [{epoch:5d}/{epochs}], Loss_train: {loss_train.item():8.4f}, Loss_test: {loss_test.item():8.4f}')

Epoch [    0/10000], Loss_train: 447.8824, Loss_test: 396.9645
Epoch [  500/10000], Loss_train:  41.7021, Loss_test:  56.1146
Epoch [ 1000/10000], Loss_train:  29.5378, Loss_test:  53.8859
Epoch [ 1500/10000], Loss_train:  27.6516, Loss_test:  53.4421
Epoch [ 2000/10000], Loss_train:  26.6496, Loss_test:  53.2362
Epoch [ 2500/10000], Loss_train:  25.8665, Loss_test:  53.0660
Epoch [ 3000/10000], Loss_train:  25.1679, Loss_test:  52.7134
Epoch [ 3500/10000], Loss_train:  24.5444, Loss_test:  52.1324
Epoch [ 4000/10000], Loss_train:  23.9913, Loss_test:  51.9851
Epoch [ 4500/10000], Loss_train:  23.3646, Loss_test:  52.3090
Epoch [ 5000/10000], Loss_train:  22.6539, Loss_test:  52.7480
Epoch [ 5500/10000], Loss_train:  21.5397, Loss_test:  53.2540
Epoch [ 6000/10000], Loss_train:  20.3302, Loss_test:  53.5728
Epoch [ 6500/10000], Loss_train:  19.1789, Loss_test:  53.3687
Epoch [ 7000/10000], Loss_train:  17.8512, Loss_test:  52.9843
Epoch [ 7500/10000], Loss_train:  16.3978, Loss_test:  